In [ ]:
import math
import tensorflow
import time
import cv2
import mss
import numpy as np
from PIL import Image
from keras.models import Sequential
from keras.models import load_model
from keras.utils import plot_model
from keras.layers import Conv2D, Dense, MaxPooling2D,Flatten,Input
from keras.optimizers import *
import tensorflow as tf
from keras import backend as K
import os

import ctypes
from ctypes import wintypes, c_int, byref
import time, random

user32 = ctypes.WinDLL('user32', use_last_error=True)

INPUT_MOUSE    = 0
INPUT_KEYBOARD = 1
INPUT_HARDWARE = 2

KEYEVENTF_EXTENDEDKEY = 0x0001
KEYEVENTF_KEYUP       = 0x0002
KEYEVENTF_UNICODE     = 0x0004
KEYEVENTF_SCANCODE    = 0x0008

MAPVK_VK_TO_VSC = 0

# msdn.microsoft.com/en-us/library/dd375731
VK_LMENU    = 0x12 # Left Alt key 18
VK_LCONTROL = 0xA2 # Left Ctrl key 162

# Playstation valid buttons
DPAD_UP     = 0x57 # W key 87
DPAD_DOWN   = 0x53 # S key 83
DPAD_LEFT   = 0x41 # A key 65
DPAD_RIGHT  = 0x44 # D key 68
CROSS       = 0x45 # E key 69
SQUARE      = 0x52 # R key 82
CIRCLE      = 0x54 # T key 84
TRIANGLE    = 0x59 # Y key 89
R1          = 0x51 # Q key 81
TOUCHPAD    = 0x58 # X l1 key 88

# Diagonal directional buttons with arbitrary values
DIAG_DOWN_LEFT = (DPAD_LEFT, DPAD_DOWN)
DIAG_DOWN_RIGHT = (DPAD_RIGHT, DPAD_DOWN)
DIAG_UP_LEFT = (DPAD_LEFT, DPAD_UP)
DIAG_UP_RIGHT = (DPAD_RIGHT, DPAD_UP)

# Multi button attacks with arbitrary values
CROSS_SQUARE = (0,(CROSS, SQUARE))
CROSS_CIRCLE = (TRIANGLE,TRIANGLE,TRIANGLE,TRIANGLE)
SQUARE_TRIANGLE = (CROSS,TRIANGLE)
TRIANGLE_CIRCLE = (0,(TRIANGLE, CIRCLE))


ATTACK01 = (SQUARE,TRIANGLE,DPAD_RIGHT,DPAD_RIGHT,TRIANGLE,DPAD_RIGHT,CIRCLE)


delay = ['hold','tap']
# available inputs by type
direction = [0, DPAD_UP, DPAD_DOWN, DPAD_LEFT, DPAD_RIGHT, DIAG_DOWN_LEFT, DIAG_DOWN_RIGHT, DIAG_UP_LEFT, DIAG_UP_RIGHT]
attack = [ATTACK01,TRIANGLE, CIRCLE, CROSS, SQUARE,0, DPAD_UP, DPAD_DOWN, DPAD_LEFT, DPAD_RIGHT, DIAG_DOWN_LEFT, DIAG_DOWN_RIGHT, DIAG_UP_LEFT, DIAG_UP_RIGHT,
           CROSS_SQUARE, CROSS_CIRCLE, SQUARE_TRIANGLE, TRIANGLE_CIRCLE,R1,]
rage = [R1]

# all valid actions
valid_actions = attack
# valid_actions = [(x,y) for x in direction for y in attack]

wintypes.ULONG_PTR = wintypes.WPARAM

class MOUSEINPUT(ctypes.Structure):
    _fields_ = (("dx",          wintypes.LONG),
                ("dy",          wintypes.LONG),
                ("mouseData",   wintypes.DWORD),
                ("dwFlags",     wintypes.DWORD),
                ("time",        wintypes.DWORD),
                ("dwExtraInfo", wintypes.ULONG_PTR))

class KEYBDINPUT(ctypes.Structure):
    _fields_ = (("wVk",         wintypes.WORD),
                ("wScan",       wintypes.WORD),
                ("dwFlags",     wintypes.DWORD),
                ("time",        wintypes.DWORD),
                ("dwExtraInfo", wintypes.ULONG_PTR))

    

    def __init__(self, *args, **kwds):
        super(KEYBDINPUT, self).__init__(*args, **kwds)
        # some programs use the scan code even if KEYEVENTF_SCANCODE
        # isn't set in dwFflags, so attempt to map the correct code.
        if not self.dwFlags & KEYEVENTF_UNICODE:
            self.wScan = user32.MapVirtualKeyExW(self.wVk,
                                                 MAPVK_VK_TO_VSC, 0)
class HARDWAREINPUT(ctypes.Structure):
    _fields_ = (("uMsg",    wintypes.DWORD),
                ("wParamL", wintypes.WORD),
                ("wParamH", wintypes.WORD))

class INPUT(ctypes.Structure):
    class _INPUT(ctypes.Union):
        _fields_ = (("ki", KEYBDINPUT),
                    ("mi", MOUSEINPUT),
                    ("hi", HARDWAREINPUT))
    _anonymous_ = ("_input",)
    _fields_ = (("type",   wintypes.DWORD),
                ("_input", _INPUT))

def _check_count(result, func, args):
    if result == 0:
        raise ctypes.WinError(ctypes.get_last_error())
    return args

class InputHandler:

    LPINPUT = ctypes.POINTER(INPUT)
    user32.SendInput.errcheck = _check_count
    user32.SendInput.argtypes = (wintypes.UINT, # nInputs
                                 LPINPUT,       # pInputs
                                 ctypes.c_int)  # cbSize

    def __init__(self):
        self.PS4RemotePlayHWND = 0
        self.PS4RemotePlayPID = 0
# Functions

    def get_actions(self, amount):
        actions = []
        actions.append([])
        action = 0
        for i in range(0,amount):
            temp = random.randint(0,1)
            if temp == 0:
                # select something random from the direction arroy
                action = direction[random.randint(0,len(direction)-1)]
            else:
                # select something random from the attack array
                action = attack[random.randint(0,len(direction)-1)]
            # Only add the input if it is not 0. 0 Is the same as nothing.
            if action != 0:
                actions.append(action)
        # Get the delay time for pressing these keys
        delayVal = delay[random.randint(0,1)]
        if delayVal == 'hold':
            # can't use i as the index because I am only adding non 0 inputs
            actions[0].append(random.uniform(0.04, 0.06))
        else:
            # can't use i as the index because I am only adding non 0 inputs
            actions[0].append(random.uniform(0.02, 0.04))
        return actions

    def get_action(self, index):
        print('Valid Actions:',valid_actions[index])
        return valid_actions[index]

    def get_remote_play_pid(self):
        # register winapi functions
        EnumWindows = ctypes.windll.user32.EnumWindows
        EnumWindowsProc = ctypes.WINFUNCTYPE(ctypes.c_bool, ctypes.POINTER(ctypes.c_int), ctypes.POINTER(ctypes.c_int))
        GetWindowText = ctypes.windll.user32.GetWindowTextW
        GetWindowTextLength = ctypes.windll.user32.GetWindowTextLengthW
        IsWindowVisible = ctypes.windll.user32.IsWindowVisible
        GetWindowThreadProcessId = ctypes.windll.user32.GetWindowThreadProcessId

        def foreach_window(self, hwnd, lParam):
            # window must be visible
            if IsWindowVisible(hwnd):
                length = GetWindowTextLength(hwnd)
                buff = ctypes.create_unicode_buffer(length + 1)
                GetWindowText(hwnd, buff, length + 1)
                try:
                    windowtitle = buff.value
                    if "REPL4Y" in windowtitle:
                        # get the processid from the hwnd
                        # declaring this as global means refer to the global version
                        #global global_PS4RemotePlayPID
                        #global global_PS4RemotePlayHWND
                        processID = c_int()
                        threadID = GetWindowThreadProcessId(hwnd, byref(processID))
                        # found the process ID
                        self.PS4RemotePlayPID = processID
                        self.PS4RemotePlayHWND = hwnd
                        return True
                except:
                    print("Unexpected error:"+sys.exc_info()[0])
                    pass;
            return True
        EnumWindows(EnumWindowsProc(foreach_window(self)), 0)


    def press_key(self, hexKeyCode):
        print(f"Pressing key: {hexKeyCode}")
        x = INPUT(type=INPUT_KEYBOARD,
                  ki=KEYBDINPUT(wVk=hexKeyCode))
        user32.SendInput(1, ctypes.byref(x), ctypes.sizeof(x))

    def release_key(self, hexKeyCode):
        print(f"Releasing key: {hexKeyCode}")
        x = INPUT(type=INPUT_KEYBOARD,
                  ki=KEYBDINPUT(wVk=hexKeyCode,
                                dwFlags=KEYEVENTF_KEYUP))
        user32.SendInput(1, ctypes.byref(x), ctypes.sizeof(x))

    def focus_window(self, hwnd):
        ctypes.windll.user32.SetForegroundWindow(hwnd)
        activate_remap()

    def activate_remap(self):
        time.sleep(0.5)
        self.press_key(VK_LCONTROL)
        self.press_key(VK_LMENU)
        time.sleep(0.01)
        self.release_key(VK_LCONTROL)
        self.release_key(VK_LMENU)
        time.sleep(0.01)
        
    def hold_delay(self):
        time.sleep(random.uniform(0.3, 0.35))

    # def quick_press_delay(self):
    #     time.sleep(random.uniform(0.03, 0.05))

    def process_keys(self, keys, action_type):
        """Recursively process key presses and releases."""
        if isinstance(keys, tuple):
            for key in keys:
                self.process_keys(key, action_type)
        else:
            if keys != 0:
                if action_type == 'press':
                    self.press_key(keys)
                elif action_type == 'release':
                    self.release_key(keys)

    def execute_action(self, actionIndex):
        action = valid_actions[actionIndex]
        print(f"Executing action: {action}")  # Debug print
        self.press_key(88)
        if(actionIndex==0): #(82, 89, 68, 68, 89, 68, 84)
            self.press_key(88)
            self.press_key(82)
            time.sleep(random.uniform(0.3, 0.35))
            self.release_key(82)
            self.press_key(89)
            time.sleep(random.uniform(0.3, 0.35))
            self.release_key(89)
            self.press_key(68)
            time.sleep(random.uniform(0.3, 0.35))
            self.release_key(68)
            self.press_key(68)
            time.sleep(random.uniform(0.3, 0.35))
            self.press_key(89)
            time.sleep(random.uniform(0.3, 0.35))
            self.release_key(89)
            self.press_key(68)
            time.sleep(random.uniform(0.03, 0.05))
            self.release_key(68)
            self.press_key(68)
            self.press_key(84)
            time.sleep(random.uniform(0.3, 0.35))
            self.release_key(84)
        # Check if action is a tuple or list
        elif isinstance(action, (tuple, list)):
            self.press_key(88)
            for key_sequence in action:
                # Press keys in the current sequence
                self.process_keys(key_sequence, 'press')
                # Hold delay after pressing keys
                self.hold_delay()
                # Release keys in the current sequence
                self.process_keys(key_sequence, 'release')
        else:
            # Handle single key press action
            if action != 0:
                self.process_keys(action, 'press')
                self.hold_delay()
                self.process_keys(action, 'release')
            else:
                print(f"Invalid action structure: {action}")
        self.release_key(88)

    def execute_actions(self, actions):
        printf("in execute_Actions",actions)
        for action in actions:
            self.execute_action(action)
            # time.sleep(random.uniform(0.01, 0.03))

    def choose_random_inputs():
        x = 100
        while x > 0:
            amount = random.randint(0,7)
            actions = get_actions(amount=amount)
            execute_actions(actions=actions)
            x = x-1
    
# This SumTree is used to create the prioritized experience replay for the DQQN.

import numpy

class SumTree:
    write = 0

    def __init__(self, capacity):
        self.capacity = capacity
        self.tree = numpy.zeros( 2*capacity - 1 )
        self.data = numpy.zeros( capacity, dtype=object )

    def _propagate(self, idx, change):
        parent = (idx - 1) // 2

        self.tree[parent] += change

        if parent != 0:
            self._propagate(parent, change)

    def _retrieve(self, idx, s):
        left = 2 * idx + 1
        right = left + 1

        if left >= len(self.tree):
            return idx

        if s <= self.tree[left]:
            return self._retrieve(left, s)
        else:
            return self._retrieve(right, s-self.tree[left])

    def total(self):
        return self.tree[0]

    def add(self, p, data):
        idx = self.write + self.capacity - 1

        self.data[self.write] = data
        self.update(idx, p)

        self.write += 1
        if self.write >= self.capacity:
            self.write = 0

    def update(self, idx, p):
        change = p - self.tree[idx]

        self.tree[idx] = p
        self._propagate(idx, change)

    def get(self, s):
        idx = self._retrieve(0, s)
        dataIdx = idx - self.capacity + 1

        return (idx, self.tree[idx], self.data[dataIdx])    
    

IMAGE_STACK = 2
IMAGE_WIDTH = 84
IMAGE_HEIGHT = 84
CHANNELS = 1
HUBER_LOSS = 2.0
LEARNING_RATE = 0.00025
def huber_loss(y_true, y_pred):
    error = y_true - y_pred
    cond = tf.abs(error) < HUBER_LOSS

    squared_loss = 0.5 * tf.square(error)
    linear_loss = HUBER_LOSS * (tf.abs(error) - 0.5 * HUBER_LOSS)

    loss = tf.where(cond, squared_loss, linear_loss)
    return tf.reduce_mean(loss)


class Model:

    def __init__(self, input_shape, actionCnt, model=None, target_model=None):
        self.input_shape = input_shape
        self.actionCnt = actionCnt

        if model is not None: 
            self.model = model
            self.target_model = target_model
        else:
            self.model = self._createModel()
            self.target_model = self._createModel()

    def _createModel(self):
        model = Sequential()
        print("Input Shape:", self.input_shape)
        print("Input Shape:", self.actionCnt)
        model.add(Conv2D(64, (2, 2), strides=(1, 1), activation='relu', padding='SAME',
                         input_shape=self.input_shape, data_format='channels_last'))

        model.add(Conv2D(32, (2,2), strides=(1,1), activation='relu'))
        model.add(Flatten())
        model.add(Dense(units=218, activation='relu'))
        model.add(Dense(units=self.actionCnt, activation='linear'))

        opt = RMSprop(learning_rate=LEARNING_RATE)
        model.compile(loss=huber_loss, optimizer=opt)
        print("Before model.summary()")
        model.summary()
        print("After model.summary()")
        return model

    def train(self, x, y, epochs=1, verbose=0):
        self.model.fit(x, y, batch_size=32, epochs=epochs, verbose=verbose)
        
    def predict(self, s, target=False):
        if target:
            return self.target_model.predict(s)
        else:
            return self.model.predict(s)

    def predict_one(self, s, target=False):
        return self.predict(s.reshape(1, IMAGE_STACK, IMAGE_WIDTH, IMAGE_HEIGHT), target).flatten()

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

MEMORY_CAPACITY = 400000
BATCH_SIZE = 32
GAMMA = 0.99
MAX_EPSILON = 1
MIN_EPSILON = 0.1

EXPLORATION_STOP = 500000
LAMBDA = - math.log(0.01) / EXPLORATION_STOP

UPDATE_TARGET_FREQUENCY = 10000

class LearningAgent:
    steps = 0
    latest_Q = 0
    epsilon = MAX_EPSILON

    def __init__(self, learning=False, epsilon=1.0, alpha=0.5):
        #super(LearningAgent, self).init()
        self.input_handler = InputHandler()
        self.learning = learning
        self.inputShape = (IMAGE_STACK, IMAGE_WIDTH, IMAGE_HEIGHT)
        self.numActions = len(valid_actions)
        self.model = Model(self.inputShape, self.numActions)
        self.memory = Memory(MEMORY_CAPACITY)
        #self.valid_actions = []
    def observe(self, sample): #(s, a, r, s_)
        x, y, errors = self.get_targets([(0, sample)])
        self.memory.add(errors[0], sample)

        if self.steps % UPDATE_TARGET_FREQUENCY == 0:
            self.model.update_target_model()

        self.steps = self.steps + 1
        self.epsilon = MIN_EPSILON + (MAX_EPSILON - MIN_EPSILON) * math.exp(-LAMBDA * self.steps)

    def get_targets(self, batch):
        no_state = np.zeros(self.inputShape)

        states = np.array([o[1][0] for o in batch])
        states_ = np.array([(no_state if o[1][3] is None else o[1][3]) for o in batch])

        # p is the prediction for a given state
        p = agent.model.predict(states)

        p_ = agent.model.predict(states_, target=False) # target set to false means predict from model
        pTarget_ = agent.model.predict(states_, target=True) # target true means predict from target model

        x = np.zeros((len(batch), IMAGE_STACK, IMAGE_WIDTH, IMAGE_HEIGHT))
        y = np.zeros((len(batch), self.numActions))
        errors = np.zeros(len(batch))

        for i in range(len(batch)):
            o = batch[i][1]
            s = o[0]; a = o[1]; r = o[2]; s_ = o[3]
            t = p[i]
            oldVal = t[a]
            if s_ is None:
                t[a] = r
            else:
                t[a] = r + GAMMA * pTarget_[i][np.argmax(p_[i])] # DDQN portion

            x[i] = s
            y[i] = t
            errors[i] = abs(oldVal - t[a])
            self.latest_Q = t[a]

        return (x, y, errors)

    def choose_action(self, state):
        # call when the agent must make a decision based on the state
        self.state = state
        self.steps += 1
        action = None

        if self.learning == False:
            #action = self.input_handler.get_action(random.randint(0,len(valid_actions)-1))
            action = random.randint(0, len(valid_actions)-1)
        else:
            if random.uniform(0,1) < self.epsilon:
                #action = self.input_handler.get_action(random.randint(0,len(valid_actions)-1))
                action = random.randint(0, len(valid_actions)-1)
                print("in if valid_actions",len(valid_actions))
            else:
                action = np.argmax(self.model.predict_one(state))
                print("in else valid_actions",len(valid_actions))
        return action

    def execute_action(self, action):
        self.input_handler.execute_action(action)

    def replay(self):
        batch = self.memory.sample(BATCH_SIZE)
        x, y, errors = self.get_targets(batch)

        for i in range(len(batch)):
            idx = batch[i][0]
            self.memory.update(idx, errors[i])

        self.model.train(x, y)

    def play(self, state):
        self.state = state
        action = np.argmax(self.model.predict_one(state))
        return action
class Vision:
    screen = {'top': 110, 'left': 0, 'width': 1920, 'height': 970}
    leftHPCapture = {'top': 110, 'left': 240, 'width': 620, 'height': 20}
    rightHPCapture = {'top': 110, 'left': 1043, 'width': 620, 'height': 20}
    
    positive = 1    # AI hit the opponent
    negative = -1   # AI took a hit

    def __init__(self, side):
        self.side = side
        with mss.mss() as sct:
            self.prevLeftHP = self.numpy_img_to_gray(np.array(sct.grab(self.leftHPCapture)))
            self.prevRightHP = self.numpy_img_to_gray(np.array(sct.grab(self.rightHPCapture)))

# Dot product to reduce the pixel values to their grayscale equivlalent
    def numpy_img_to_gray(self, img):
        return np.dot(img[...,:3], [0.299,0.587,0.114])

    def get_current_screen(self):
        with mss.mss() as sct:
            sct_img = sct.grab(self.screen)
            img = Image.frombytes('RGB', sct_img.size, sct_img.rgb)
            img = img.resize((IMAGE_WIDTH, IMAGE_HEIGHT), Image.LANCZOS)
            currScreen = np.array(img)
        return self.numpy_img_to_gray(currScreen)

    def get_reward(self):
        with mss.mss() as sct:
            reward = 0

            currLeftHP = np.array(sct.grab(self.leftHPCapture))
            currRightHP = np.array(sct.grab(self.rightHPCapture))
            # Convert to gray
            currLeftHP = self.numpy_img_to_gray(currLeftHP)
            currRightHP = self.numpy_img_to_gray(currRightHP)
            # get the difference in previous vs current
            diffLeftHP = self.prevLeftHP - currLeftHP
            diffRightHP = self.prevRightHP - currRightHP
            # round negative values up to 0
            diffLeftHP = diffLeftHP.clip(min=0)
            diffRightHP = diffRightHP.clip(min=0)
            # If hit, there are typically more than 10 pixels with values > 120
            if((diffLeftHP > 125).sum() > 10):
                if(self.side == 'left'):
                    reward = reward - 1
                else:
                    reward = reward + 1
            # If hit, there are typically more than 10 pixels with values > 120
            if((diffRightHP > 125).sum() > 10):
                if(self.side == 'right'):
                    reward = reward - 1
                else:
                    reward = reward + 1
            # Set previous frame data to current frame data
            self.prevLeftHP = currLeftHP
            self.prevRightHP = currRightHP

        return reward

# SumTree of previous decisions
class Memory:
    def __init__(self, capacity, epsilon=1.0, alpha=0.5):
        self.tree = SumTree(capacity)
        self.epsilon = epsilon
        self.alpha = alpha

    def _getPriority(self, error):
        return (error + self.epsilon) ** self.alpha

    def add(self, error, sample):
        p = self._getPriority(error)
        self.tree.add(p, sample)

    def sample(self, n):
        batch = []
        segment = self.tree.total() / n

        for i in range(n):
            a = segment * i
            b = segment * (i + 1)

            s = random.uniform(a, b)
            (idx, p, data) = self.tree.get(s)
            batch.append((idx, data))

        return batch

    def update(self, idx, error):
        p = self._getPriority(error)
        self.tree.update(idx, p)

def import_model(agent):
    try:
        agent.model = Model(agent.inputShape, agent.numActions,
            load_model('model backup/trial 2/TekkenBotDDQN_1.h5', custom_objects={'huber_loss':huber_loss}),
                            load_model('model backup/trial 2/TekkenBotDDQN_Target_1.h5', custom_objects={'huber_loss':huber_loss}))
        print('Model loaded for agent.')
    except Exception as e:
        print('Model failed to load.')
        print(e)

# Used when not learning.
def play(agent):
    agent.input_handler.activate_remap()
    vision = Vision('left')
    w = vision.get_current_screen()
    state = np.array([w,w])
    try:
        while True:
            actionIndex = agent.play(state)
            agent.execute_action(actionIndex)
            screenCap = vision.get_current_screen()
            statePrime = np.array((state[1], screenCap))
            state = statePrime

    except KeyboardInterrupt:
        print('break')


TRIAL = 1 # necessary for correct model backup
TOTAL_TESTS = 1 # 3 12 hour segments of testing
TOTAL_EPISODES =  100 # 720 = 12 hours
def save_models(agent, curr_test):

    directory = 'model backup/trial 2'
    if not os.path.exists(directory):
        os.makedirs(directory)
    filename = directory + "/TekkenBotDDQN_1.h5"
    agent.model.model.save(filename, overwrite=True)
    print("Model saved as: TekkenBotDDQN_1.h5")

    filename = directory + "/TekkenBotDDQN_Target_1.h5"
    agent.model.target_model.save(filename, overwrite=True)
    print("Target Model saved as: TekkenBotDDQN_Target_1.h5")

# lets start the show. Used when learning.
def run(agent):
    agent.input_handler.activate_remap()
    vision = Vision('left')
    w = vision.get_current_screen()
    state = np.array([w, w])
    start_time = time.time()

    rewardTotal = 0
    episode = 0
    true_episode = 0
    curr_test = 1
    record_size = TOTAL_EPISODES * TOTAL_TESTS + 1
    reward_per_episode = np.zeros((record_size, 4))
    i = 0

    try:
        while True:
            actionIndex = agent.choose_action(state)
            agent.execute_action(actionIndex)
            screenCap = vision.get_current_screen()
            reward = vision.get_reward()

            statePrime = np.array((state[1], screenCap))

            agent.observe((state, actionIndex, reward, statePrime))
            agent.replay()

            state = statePrime
            rewardTotal += reward

            if time.time() - start_time > 59:
                reward_per_episode[i] = [true_episode, rewardTotal, agent.steps, agent.latest_Q]
                print("Episode {} Ended. Reward earned: {}".format(episode, rewardTotal))
                episode += 1
                true_episode += 1
                rewardTotal = 0
                i += 1
                start_time = time.time()

                # Perform evaluation every 10 episodes (or any interval you prefer)
                if episode % 2 == 0:#10
                    evaluate(agent, num_episodes=2)#10

            if episode >= TOTAL_EPISODES:
                if curr_test >= TOTAL_TESTS:
                    save_models(agent, curr_test)
                    agent.input_handler.activate_remap()
                    raise KeyboardInterrupt('Episodes limit reached')
                else:
                    save_models(agent, curr_test)
                    curr_test += 1
                    episode = 0
                    start_time = time.time()

    except KeyboardInterrupt:
        np.savetxt('model backup/trial 2/episodesAndRewards.txt'.format(TRIAL), reward_per_episode, fmt='%d')
        print("Episodes and rewards saved to episodesAndRewards.txt")
        save_models(agent, curr_test)
        
def evaluate(agent, num_episodes=2):#10
    agent.input_handler.activate_remap()
    vision = Vision('left')
    rewards = []
    episode_lengths = []

    for episode in range(num_episodes):
        obs = vision.get_current_screen()
        state = np.array([obs, obs])
        done = False
        total_reward = 0
        steps = 0
        start_time = time.time()  # Initialize start_time for each episode

        while not done:
            actionIndex = agent.choose_action(state)
            agent.execute_action(actionIndex)
            screenCap = vision.get_current_screen()
            reward = vision.get_reward()

            statePrime = np.array((state[1], screenCap))
            state = statePrime

            total_reward += reward
            steps += 1

            # Determine the end of an episode based on your specific conditions
            if time.time() - start_time > 59:  # Or other episode-ending conditions
                done = True

        rewards.append(total_reward)
        episode_lengths.append(steps)

    mean_reward = np.mean(rewards)
    std_reward = np.std(rewards)
    mean_length = np.mean(episode_lengths)
    std_length = np.std(episode_lengths)

    print(f'Evaluation over {num_episodes} episodes:')
    print(f'Average Reward: {mean_reward} ± {std_reward}')
    print(f'Average Episode Length: {mean_length} ± {std_length}')
    
    return rewards, episode_lengths

if __name__ == '__main__':
    try:
        agent = LearningAgent(learning=True, epsilon=MAX_EPSILON, alpha=LEARNING_RATE)
        import_model(agent)
        run(agent)
        # play(agent)
    finally:
        print('Thanks for playing!')